In [2]:
# import dependencies
import requests
import json
from pprint import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# connect to API key
from config import gkey


In [3]:
# read zip codes csv
zipcodes_data = "ny_state_zipcodes.csv" 
zipcodes_df = pd.read_csv(zipcodes_data)
zipcodes_df.head()
#print(len(zipcodes_df))


,Zip Code
0,10001
1,10002
2,10003
3,10004
4,10005


In [4]:
# convert zip codes from integers to string
zipcodes_df = zipcodes_df.astype({"Zip Code": str})
zipcodes_df.dtypes


Zip Code    object
dtype: object

In [5]:
# output the column of zip codes into a list and hold in a variable for NY zip codes
ny_zip_codes = zipcodes_df["Zip Code"].tolist()
#print(zip_codes)
len(ny_zip_codes)

# !!!!!!!!!  notice we have 2,202 zip codes for NY state   !!!!!!!!!!!!!!


2202

In [6]:
# establish base URL 
base_url = "https://api.census.gov/data/2017/acs/acs5/profile"

# print the response to be sure it's connecting successfully
response = requests.get(base_url)
print(response)


<Response [200]>


In [7]:
# identify and set variables for keys from Census that we want to call
avg_income = "DP03_0063E"   
single_hh = "DP02_0011E"
hh_no_vehicles = "DP04_0058E"
#chinese_pop = "DP05_0046E"
zip_code = "zip%20code%20tabulation%20area:*"

#build query URL
query_url = f"{base_url}?get={avg_income},{single_hh},{hh_no_vehicles},NAME&for={zip_code}&key={gkey}"

#alternate query URL if we want to check specific zip codes (example for zip 10007 below):
#query_url = f"{base_url}?get={avg_income},{single_hh},{hh_no_vehicles},NAME&for=zip%20code%20tabulation%20area:10006&key={gkey}"

print(query_url)


https://api.census.gov/data/2017/acs/acs5/profile?get=DP03_0063E,DP02_0011E,DP04_0058E,NAME&for=zip%20code%20tabulation%20area:*&key=391a26e50b873d297dfed1954142d1f1dee85f9e


In [8]:
# convert to json
response = requests.get(query_url).json()
pprint(response)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [126]:
# create a dataframe containing data above (all US zip codes)

# set a column name for each output from JSON response above
df_raw = pd.DataFrame(response, columns=['Avg Income ($)', 
                                         'Single HH', 
                                         'No Vehicle HH',
                                         'ZCTA', 
                                         'Zip Code'])

# re-order columns and eliminate redundant columns 
df_raw = df_raw[['Zip Code',
                 'Avg Income ($)',
                 'Single HH', 
                 'No Vehicle HH']]

# drop first row, as it's a header
df_raw = df_raw.drop([0], axis=0, inplace= False)

df_raw

# !!!!!!!!!!! notice there are 33,120 US zips  !!!!!!!!!!!!!!!!!!!


,Zip Code,Avg Income ($),Single HH,No Vehicle HH
1,01379,66682,118,20
2,01440,62384,2461,915
3,01505,119858,398,43
4,01524,86771,738,100
5,01529,104300,207,40
6,01583,100363,539,84
7,01588,84802,1033,230
8,01609,77570,3276,1771
9,01721,136132,1424,145
10,01731,98165,17,0


In [132]:
# see what data types we have
df_raw.dtypes


Zip Code          object
Avg Income ($)    object
Single HH         object
No Vehicle HH     object
dtype: object

In [139]:
# convert data types where necessary to floats
df_raw = df_raw.astype({"Zip Code": str,"Avg Income ($)": float, "Single HH": float, "No Vehicle HH": float})
df_raw.dtypes



Zip Code           object
Avg Income ($)    float64
Single HH         float64
No Vehicle HH     float64
dtype: object

In [138]:
# remove any negative numbers

#us_df = df_raw[(df_raw["Avg Income ($)"]>=0) & 
 #           (df_raw["Single HH"]>=0) & 
 #           (df_raw["No Vehicle HH"]>=0)]

#us_df['Zip Code']=us_df['Zip Code'].apply(lambda x: '{0:0>5}'.format(x))

#us_df.head()
#print(len(us_df)) = 31814

#  !!!!!!!!!!!!!! notice that after dropping zips w negative data, now at 31,814 zips.  1,306 were lost  !!!!!!!!!

31814


In [146]:
# create a dataframe with just NY zip codes

ny_df = df_raw.loc[df_raw['Zip Code'].isin(zip_codes)]
print(len(ny_df))
ny_df.head()

# !!!!!!! notice only 1,791 of the 2,202 NY zips were located in our US zip census data !!!!!!!!!!! 


1791


,Zip Code,Avg Income ($),Single HH,No Vehicle HH
2555,11548,194906.0,39.0,26.0
2556,11558,108979.0,667.0,300.0
2557,11560,206151.0,542.0,46.0
2558,11579,152379.0,433.0,91.0
2559,11694,104387.0,3152.0,1944.0


In [148]:
# run some calculations to understand the data

print(ny_df["Avg Income ($)"].count())
print(ny_df["Avg Income ($)"].median())
print(ny_df["Avg Income ($)"].mean())
print("------------------------------------")
print(ny_df.mean())

# !!!!!!!!! notice  there must be huge incorrect negative numbers in income data  !!!!!!!!


1791
71179.0
-36955005.11111111
------------------------------------
Zip Code                   inf
Avg Income ($)   -3.695501e+07
Single HH         1.205048e+03
No Vehicle HH     1.172084e+03
dtype: float64


In [154]:
# remove any negative numbers
ny_df_clean = ny_df[(ny_df["Avg Income ($)"]>=0) & 
                    (ny_df["Single HH"]>=0) & 
                    (ny_df["No Vehicle HH"]>=0)]

#us_df['Zip Code']=us_df['Zip Code'].apply(lambda x: '{0:0>5}'.format(x))

ny_df_clean

# !!!!!!! notice that after dropping zips w negative data, now at 1707 zips. 84 were lost !!!


,Zip Code,Avg Income ($),Single HH,No Vehicle HH
2555,11548,194906.0,39.0,26.0
2556,11558,108979.0,667.0,300.0
2557,11560,206151.0,542.0,46.0
2558,11579,152379.0,433.0,91.0
2559,11694,104387.0,3152.0,1944.0
2560,11697,124884.0,387.0,149.0
2561,11703,110001.0,1168.0,188.0
2562,11729,100999.0,1926.0,474.0
2563,11732,161947.0,477.0,41.0
2564,11747,166217.0,1585.0,179.0


In [1]:
# create list of remaining NY zip codes to cross reference in other APIs

operational_ny_zip_codes = ny_df_clean["Zip Code"].tolist()
#len(operational_ny_zip_codes)
#operational_ny_zip_codes

ny_final_zips_df = pd.DataFrame(operational_ny_zip_codes)
ny_final_zips_df 


NameError: name 'ny_df_clean' is not defined